In [1]:

# %tensorflow_version 1.x
# import tensorflow as tf
# from tensorflow import keras

In [1]:
! pip uninstall -y tensorflow
! pip install -q tf-nightly
! pip install -q tensorflow-model-optimization
import tensorflow as tf
from tensorflow import keras


Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 394.8MB 42kB/s 
     |████████████████████████████████| 3.8MB 54.3MB/s 
     |████████████████████████████████| 1.3MB 48.8MB/s 
     |████████████████████████████████| 471kB 61.1MB/s 
     |████████████████████████████████| 14.5MB 250kB/s 
     |████████████████████████████████| 10.6MB 57.7MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 174kB 4.4MB/s 


Reference Link :https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md

Future Useful Links
1. https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide.md
2.https://colab.research.google.com/gist/ohtaman/c1cf119c463fd94b0da50feea320ba1e/edgetpu-with-keras.ipynb#scrollTo=jWp9_I06ZjDo
2.https://paperswithcode.com/paper/quantization-and-training-of-neural-networks

In [2]:
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# train_images=train_images.reshape(60000, 28, 28)
# test_images=test_images.reshape(10000, 28, 28)

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0
# Batches of 64
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(64)

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from tensorflow import Tensor
def sgcn_block(x:Tensor,kernel_size: int):

  x = keras.layers.DepthwiseConv2D(kernel_size=(1, 11),padding='same',depth_multiplier=1)(x)
  x1 = keras.layers.Conv2D(190,(1,1))(x)
  x1 = keras.layers.BatchNormalization(fused=False)(x1)
  x1 = keras.layers.Activation('relu')(x1)
  # x1 = keras.layers.ReLU()(x1)

  x2 = keras.layers.Conv2D(190,(1,1))(x)
  x2 = keras.layers.BatchNormalization(fused=False)(x2)
  x2 = keras.layers.Activation('sigmoid')(x2)

  out=keras.layers.Multiply()([x1,x2])
  # quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
  print(out.shape)
  return out

def create_sgcn():
  inputs=keras.Input(shape=(28, 28,1))
  num_filters=64 #how many?

  # t=keras.layers.BatchNormalization(fused=False)(inputs)
  t=tf.keras.layers.Conv2D(num_filters,5)(inputs)
  t=tf.keras.layers.MaxPool2D((2,2))(t)
  t=keras.layers.Activation('relu')(t)
  t=tf.keras.layers.Conv2D(num_filters,5)(t)
  t=tf.keras.layers.MaxPool2D((2,2))(t)
  t=keras.layers.Activation('relu')(t)
  print("shape***",t.shape)
  s=t.shape
  t=keras.layers.Reshape(target_shape=(1,s[1]*s[2],s[3]))(t)
  # t=keras.layers.Flatten()(t) #necessary? transpose or expand dim (2nd dimension is channel)
  print("shape squeeze",t.shape)
  # input2=keras.Input(shape=(32,32))
  # t=keras.layers.Reshape(target_shape=(32,32))(t)
  # t=keras.layers.BatchNormalization(fused=False)(input2)
  for i in range(1,13): #correct?
    t=sgcn_block(t,5)
    if i%2!=0:
      print(i)
      t1=t
    if i%2 !=0 & i!=1:
      print("i:",i)
      t=keras.layers.Add()([t1,t])
  print("hi",t.shape)
  out=keras.layers.Flatten()(t)
  out= keras.layers.Dense(10, activation="softmax")(out)
  print(out.shape)
  model=keras.Model(inputs, out, name="SGCN")
  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  print(model.summary())
  return model



In [4]:
# model.compile(
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer=keras.optimizers.RMSprop(),
#     metrics=["accuracy"],
# )
model=create_sgcn()

history = model.fit(train_images, train_labels, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(test_images, test_labels, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

shape*** (None, 4, 4, 64)
shape squeeze (None, 1, 16, 64)
(None, 1, 16, 190)
1
i: 1
(None, 1, 16, 190)
(None, 1, 16, 190)
3
i: 3
(None, 1, 16, 190)
(None, 1, 16, 190)
5
i: 5
(None, 1, 16, 190)
(None, 1, 16, 190)
7
i: 7
(None, 1, 16, 190)
(None, 1, 16, 190)
9
i: 9
(None, 1, 16, 190)
(None, 1, 16, 190)
11
i: 11
(None, 1, 16, 190)
hi (None, 1, 16, 190)
(None, 10)
Model: "SGCN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 24, 24, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D) 

**Quantized Aware Training**

In [5]:
import tensorflow_model_optimization as tfmot
def apply_quantization_to_layer(layer):
  if isinstance(layer, tf.keras.layers.MaxPool2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.Conv2D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.SeparableConv1D):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.Flatten):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  if isinstance(layer, tf.keras.layers.Add):
  #   return tfmot.quantization.keras.quantize_annotate_layer(layer)
  # if isinstance(layer, tf.keras.layers.Multiply):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  # if isinstance(layer, tf.keras.layers.Activation):
  #   return tfmot.quantization.keras.quantize_annotate_layer(layer)
  
  return layer

In [6]:
# quant_aware_model = tfmot.quantization.keras.quantize_model(model)

In [7]:
#Use `tf.keras.models.clone_model` to apply #apply_quantization_to_layer` 
# to the layers of the model.

annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization_to_layer
    
)

In [8]:
# model = quantize_annotate_model(tf.keras.Sequential([
#    quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
#    tf.keras.layers.Flatten()
# ]))

# Now that the layers are annotated,
# `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
# with quantize_scope(
#   {'DefaultQuantizeConfig': DefaultQuantizeConfig):
#   # Use `quantize_apply` to actually make the model quantization aware.
#   quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Model: "SGCN"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
quantize_layer (QuantizeLayer)  (None, 28, 28, 1)    3           input_1[0][0]                    
__________________________________________________________________________________________________
quant_conv2d (QuantizeWrapper)  (None, 24, 24, 64)   1795        quantize_layer[1][0]             
__________________________________________________________________________________________________
quant_max_pooling2d (QuantizeWr (None, 12, 12, 64)   1           quant_conv2d[0][0]               
_______________________________________________________________________________________________

In [9]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)
quant_aware_model.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
quant_aware_model.fit(train_images,train_labels,epochs=1,validation_split=0.1,)

1688/1688 [==============================] - 332s 194ms/step - loss: 0.0467 - accuracy: 0.9862 - val_loss: 0.0280 - val_accuracy: 0.9913


**Convert to TFLite**

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2338: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:1395: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: /tmp/tmpb91vrs43/assets


INFO:tensorflow:Assets written to: /tmp/tmpb91vrs43/assets


In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()
with open('MNIST_QAT.tflite', 'wb') as f:
  f.write(quantized_tflite_model)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:2338: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:1395: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: /tmp/tmplfrdcyb6/assets


INFO:tensorflow:Assets written to: /tmp/tmplfrdcyb6/assets


In [22]:
import os
model.save('MNIST_base_model.h5')
print(os.stat('MNIST_base_model.h5').st_size)
print(os.stat('MNIST_QAT.tflite').st_size)

12424840
1333680


In [23]:

print("Float model in Mb:", os.path.getsize('MNIST_base_model.h5') / float(2**20))
print("Quantized model in Mb:", os.path.getsize('MNIST_QAT.tflite') / float(2**20))

Float model in Mb: 11.849250793457031
Quantized model in Mb: 1.2718963623046875


In [19]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

_, q_aware_model_accuracy = quant_aware_model.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.978600025177002
Quant test accuracy: 0.9921000003814697


**Test Accuracy on TFLite model**

In [17]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    test_image=tf.reshape(test_image,(28,28,1))
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

In [18]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9921


NameError: ignored

In [20]:
print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Quant TFLite test_accuracy: 0.9921
Quant TF test accuracy: 0.9921000003814697
